In [1]:
import re
import os
from os import walk
import pandas as pd
import csv

In [2]:
def merge_weather_client(client_df):
    weather_df = pd.read_csv("Weather/C3 uurgeg_344_2021-2030.csv")
    
    #We need to merge the client and weather dataframes on date and hour.
    #First, we manipulate the date/time data type of the client dataframe to be like that of the weather dataframe
    
    date_time_df = client_df.date.str.split(expand=True)

    time_split = date_time_df[1].str.split(':', expand = True)

    date_split = date_time_df[0].str.split('-', expand=True)
    new_date = date_split[2] + '/' + date_split[1] + '/' + date_split[0]


    client_df.insert(1, "MM", time_split[1], True)
    client_df.insert(1, "HH", time_split[0], True)
    client_df.insert(1, "YYYYMMDD", new_date, True)

    client_df.HH = client_df.HH.astype('float64')
    
        
    new_df = pd.merge(client_df, weather_df,  how='left', 
                      left_on=['YYYYMMDD', 'HH'], right_on=['DATE', '   HH'])
    
    return new_df

In [3]:
def get_calculated_measures(filename):
    df = pd.read_csv(filename)
    df.columns = ["date", "location_uuid", "t_ambient", "t_globe", "hum_ambient", "hum_globe", 
                  "co2", "tvoc", "pm1", "pm2_5", "pm10", "aerosol_small", "aerosol_large"]
    
    df = df.dropna()
    
    #temp difference
    m1 = df["t_ambient"] - df["t_globe"]

    #humidity difference
    m2 = df["hum_ambient"] - df["hum_globe"]
    
    #pm percentage measures
    pm_total = df["pm1"] + df["pm2_5"] + df["pm10"]
    pm1_perc = df["pm1"]/pm_total
    pm2_5_perc = df["pm2_5"]/pm_total
    pm10_perc = df["pm10"]/pm_total

    #delta measures
    delta_co2 = df["co2"].diff(periods = 1)
    delta_voc = df["tvoc"].diff(periods = 1)
    delta_t_ambient = df["t_ambient"].diff(periods = 1)
    
    #double derivative measures
    delta2_co2 = delta_co2.diff(periods = 1)
    delta2_voc = delta_voc.diff(periods = 1)
    delta2_t_ambient = delta_t_ambient.diff(periods = 1)
    
    #merge client and weather data before getting the diff_in_temp calc feature
    client_weather_df = merge_weather_client(df)

    #diff betweeen indoor and outdoor temp is calculated in a separate function
    diff_in_temp = client_weather_df['t_ambient'] - client_weather_df['    T']
    
    #the following is a dataset with every calculated measure and their respective date and room location
    concatenated = pd.concat([df["date"], df["location_uuid"], m1, m2, pm1_perc, pm2_5_perc, pm10_perc, 
                              delta_co2, delta_voc, delta_t_ambient, 
                              delta2_co2, delta2_voc, delta2_t_ambient,
                              diff_in_temp], axis="columns")
    
    
    concatenated.columns = ["Date", "Location", "temperature_diff", "humidity_diff",
                    "percentage_pm_1", "percentage_pm_2_5", "percentage_pm_10", "delta_co2",
                    "delta_voc", "delta_temp", "delta2_co2", "delta2_voc", "delta2_t_ambient",
                            "weather_temp_diff"]
    
    concatenated = concatenated.dropna()
    
    #the following is a dataset consisting of the merging of the client data, weather data and calculated feeatures
    final_df = pd.merge(concatenated, client_weather_df,  how='left', 
                left_on=['Date', 'Location'], right_on=['date', 'location_uuid'])
    
    print(final_df)   

    final_df.to_csv(path_or_buf="Calculated Measures/calc_measures_" + filename,
                                 sep=',', na_rep='', float_format=None, columns=None,
                                 header=True, index=True, index_label=None, mode='w',
                                 encoding=None, compression='infer', quoting=None, quotechar='"', 
                                 line_terminator=None, chunksize=None, date_format=None, 
                                 doublequote=True, escapechar=None, decimal='.')




In [5]:
os.getcwd()
os.chdir('C:\\Users\\Luke Xuereb\\Documents\\Masters\\Smart Services Project\\Data\\MU SSP Shared Folder Clairify\\Export')

filename = 'client_3.csv'
get_calculated_measures(filename)



                                    Date  \
0      2021-10-13 13:15:00.000000 +00:00   
1      2021-10-13 13:20:00.000000 +00:00   
2      2021-10-13 13:25:00.000000 +00:00   
3      2021-10-13 13:30:00.000000 +00:00   
4      2021-10-13 13:35:00.000000 +00:00   
...                                  ...   
88311  2022-01-01 08:00:00.000000 +00:00   
88312  2022-01-01 08:05:00.000000 +00:00   
88313  2022-01-01 08:10:00.000000 +00:00   
88314  2022-01-01 08:15:00.000000 +00:00   
88315  2022-01-01 08:20:00.000000 +00:00   

                                   Location  temperature_diff  humidity_diff  \
0      0eae5567-37b4-4764-916f-88baec488e03             -0.62           7.39   
1      0eae5567-37b4-4764-916f-88baec488e03             -0.60           6.55   
2      0eae5567-37b4-4764-916f-88baec488e03             -0.64           6.19   
3      0eae5567-37b4-4764-916f-88baec488e03             -0.68           5.97   
4      0eae5567-37b4-4764-916f-88baec488e03             -0.71          